In [1]:
## Imports
import cv2
import numpy as np

from utils.DisplayAnswers import showAnswers
from utils.stackImages import stackImages
from utils.rectangleDetector import detectRectangleContours
from utils.SplitImage import splitBoxes
from utils.rectangleDetector import getCornerPoints, reOrder


In [5]:
path = "img.png"
widthImg = 700
heightImg = 700
questions = 5
choices = 5
answers = [1, 2, 0, 1, 4]  # correct answers to questions

webcamFeed = False
cameraNumber = 1  # zero is for webcam

In [6]:
# Camera feeding image
capture = cv2.VideoCapture(cameraNumber)
capture.set(10, 150)  # set brightness

while True:
    if webcamFeed:
        success, img = capture.read()
    else:
        img = cv2.imread(path)

    ## Preprocessing
    img = cv2.imread(path)
    imgContours = img.copy()
    imgBiggestContours = img.copy()
    imgFinal = img.copy()
    img = cv2.resize(img, (widthImg, heightImg))
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1)
    imgCanny = cv2.Canny(imgBlur, 10, 15)


    try:
        ## FINDING ALL CONTOURS
        contours, hierarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10)

        ## FIND RECTANGLES
        rectCon = detectRectangleContours(contours)
        biggestContour = getCornerPoints(
            rectCon[0])  ## 0 means largest contour if you need second largest contour change it to 1
        print(biggestContour.shape)  ## 4,1,2
        gradePoints = getCornerPoints(rectCon[1])
        print("Biggest Con : ", biggestContour)

        if biggestContour.size != 0 and gradePoints.size != 0:
            cv2.drawContours(imgBiggestContours, biggestContour, -1, (0, 255, 0), 20)
            cv2.drawContours(imgBiggestContours, gradePoints, -1, (255, 0, 0), 20)

            biggestContour = reOrder(biggestContour)
            gradePoints = reOrder(gradePoints)

            point1 = np.float32(biggestContour)
            point2 = np.float32([[0, 0], [widthImg, 0], [0,heightImg], [widthImg, heightImg]])
            matrix = cv2.getPerspectiveTransform(point1, point2)
            imgWarpColored = cv2.warpPerspective(img, matrix, (widthImg, heightImg))

            gradePoint1 = np.float32(gradePoints)
            gradePoint2 = np.float32([[0, 0], [325, 0], [0.150], [325, 150]])
            gradeMatrix = cv2.getPerspectiveTransform(gradePoint1, gradePoint2)
            gradeImgWarpColored = cv2.warpPerspective(img, gradeMatrix, (325, 150))
            # cv2.imshow("Grade", gradeImgWarpColored)

            ## Apply threshold and find marked answer
            imgWarpGray = cv2.cvtColor(imgWarpColored, cv2.COLOR_BGR2GRAY)
            imgThreshold = cv2.threshold(imgWarpGray, 150, 255, cv2.THRESH_BINARY_INV)[1]

            boxes = splitBoxes(imgThreshold)
            # cv2.imshow("Test", boxes[2])
            # print(cv2.countNonZero(boxes[1]),cv2.countNonZero(boxes[2]))

            ## Getting non zero pixel values of each box
            myPixelValues = np.zeros((questions, choices))
            countColumn = 0
            countRow = 0

            for image in boxes:
                totalPixels = cv2.countNonZero(image)
                myPixelValues[countRow][countColumn] = totalPixels
                countColumn += 1
                if countColumn == choices:
                    countRow += 1
                    countColumn = 0

            print(myPixelValues)

            # Finding indexes of marking
            myIndex = []
            for x in range(0, questions):
                arr = myPixelValues[x]
                myIndexValue = np.where(arr == np.amax(arr))
                myIndex.append(myIndexValue[0][0])
            print(myIndex)

            # Grading
            grading = []
            for x in range(0, questions):
                if answers[x] == myIndex[x]:
                    grading.append(1)
                else:
                    grading.append(0)
            print(grading)

            # Calculate score
            score = (sum(grading) / questions) * 100
            print("Score :", score)

            # Displaying answers
            imgResult = imgWarpColored.copy()
            imgResult = showAnswers(imgResult, myIndex, grading, answers, questions, choices)

            # Extract answers to blank image
            imgRawDrawing = np.zeros_like(imgWarpColored)
            imgRawDrawing = showAnswers(imgResult, myIndex, grading, answers, questions, choices)

            inverseMatrix = cv2.getPerspectiveTransform(point2, point1)
            imgInverseWarpColored = cv2.warpPerspective(imgRawDrawing, inverseMatrix, (widthImg, heightImg))

            # Generate grade image
            imgRawGrade = np.zeros_like(gradeImgWarpColored)
            cv2.putText(imgRawGrade, str(int(score)) + "%", (60, 100), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 255, 255), 3)
            #cv2.imshow("Grade", imgRawGrade)
            inverseGradeMatrix = cv2.getPerspectiveTransform(gradePoint2, gradePoint1)
            inverseGradeImgWarpColored = cv2.warpPerspective(imgRawGrade, inverseGradeMatrix, (widthImg, heightImg))

            imgFinal = cv2.addWeighted(imgFinal, 1, imgInverseWarpColored, 1, 0)
            imgFinal = cv2.addWeighted(imgFinal, 1, inverseGradeImgWarpColored, 1, 0)

        imgBlank = np.zeros_like(img)
        imageArray = (
            [imgBlank, imgBlank, imgBlank, imgBlank],
            [imgBlank, imgBlank, imgBlank, imgBlank],
            [imgBlank, imgBlank, imgBlank, imgBlank]
        )
    except:
        imgBlank = np.zeros_like(img)
        imageArray = (
            [img, imgGray, imgBlur, imgCanny],
            [imgContours, imgBiggestContours, imgWarpColored, imgThreshold],
            [imgResult, imgRawDrawing, imgInverseWarpColored, imgFinal]
        )

    lables = [
        ["Original", "Gray", "Blur", "Canny"],
        ["Contours", "Biggest Contour", "Warp", "Threshold"],
        ["Result", "Raw Drawing", "Inverse Warp", "Final"]
    ]
    imageStacked = stackImages(0.3, imageArray, lables)

    cv2.imshow("Original", imageStacked)
    cv2.imshow("Final Result", imgFinal)
    # cv2.waitKey(0) # commented this line due to implement below feature

    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite("FinalResult.jpg", imgFinal)
        cv2.waitKey(300)


Area :  0.0
Area :  0.0
Area :  82.0
Corner points :  8
Area :  123.5
Corner points :  7
Area :  0.0
Area :  78.5
Corner points :  12
Area :  0.0
Area :  0.0
Area :  0.0
Area :  11.5
Area :  8.0
Area :  6.5
Area :  0.0
Area :  98.0
Corner points :  9
Area :  0.0
Area :  0.0
Area :  125.5
Corner points :  8
Area :  0.0
Area :  1.0
Area :  1.0
Area :  0.0
Area :  0.5
Area :  0.0
Area :  0.0
Area :  0.5
Area :  10.0
Area :  1.0
Area :  15.5
Area :  14.5
Area :  26.5
Area :  38.5
Area :  6.5
Area :  6.0
Area :  332.0
Corner points :  12
Area :  0.5
Area :  0.0
Area :  2.0
Area :  12.0
Area :  14.5
Area :  0.0
Area :  0.0
Area :  38.5
Area :  0.0
Area :  11.5
Area :  0.0
Area :  17.5
Area :  0.5
Area :  426.0
Corner points :  12
Area :  26.5
Area :  0.0
Area :  13.0
Area :  12.0
Area :  24.0
Area :  1.0
Area :  0.5
Area :  0.0
Area :  0.5
Area :  475.5
Corner points :  7
Area :  0.5
Area :  480.5
Corner points :  8
Area :  0.0
Area :  22135.0
Corner points :  4
Area :  13589.5
Corner points

NameError: name 'imgWarpColored' is not defined